In [19]:
%tensorflow_version 1.x
import pickle
import numpy as np
import tensorflow as tf
import PIL.Image
import sys
import io
import uuid
import os
from PIL import Image
from pprint import pprint
from zipfile import ZipFile
import cv2
# tf.InteractiveSession()
path="/content/drive/MyDrive/GAN/progan";
sys.path.insert(0,'/content/drive/MyDrive/GAN/progan/stylegan2-ada-main')

In [2]:
tf.__version__

'1.15.2'

In [3]:
# !pip install fastapi
# !pip install colabcode

In [4]:
!pip install flask_cors flask-ngrok

In [5]:
def file_process_in_memory(images):
    """ Converts PIL image objects into BytesIO in-memory bytes buffers. """
    image=[]
    for i,  pil_image in enumerate(images):
        file_object = io.BytesIO()
        pil_image=PIL.Image.fromarray(pil_image,'RGB')
        pil_image.save(file_object, "PNG")
        pil_image.close()
        image.append( file_object)  # Replace PIL image object with BytesIO memory buffer.

    return image  # Return modified list.


In [6]:
def generate(number_of_images):
  # tf.InteractiveSession.close(self)
  tf.InteractiveSession()
  with open(path+'/ffhq.pkl', 'rb') as file:
      Gs= pickle.load(file)[2]
  
  latents = np.random.randn(number_of_images, *Gs.input_shapes[0][1:])
  labels = np.zeros([latents.shape[0]] + Gs.input_shapes[1][1:])
  images = Gs.run(latents, labels)
  images = np.clip(np.rint((images + 1.0) / 2.0 * 255.0), 0.0, 255.0).astype(np.uint8) 
  images = images.transpose(0, 2, 3, 1)
  PIL.Image.fromarray(images[0],'RGB').save('img.jpg')

In [30]:
def generatezip(number_of_images=10,n=0):
  # tf.InteractiveSession.close(self)
  tf.InteractiveSession()
  with open(path+'/ffhq.pkl', 'rb') as file:
      Gs= pickle.load(file)[2]
  
  latents = np.random.randn(number_of_images, *Gs.input_shapes[0][1:])
  labels = np.zeros([latents.shape[0]] + Gs.input_shapes[1][1:])
  images = Gs.run(latents, labels)
  images = np.clip(np.rint((images + 1.0) / 2.0 * 255.0), 0.0, 255.0).astype(np.uint8) 
  images = images.transpose(0, 2, 3, 1)
  # PIL.Image.fromarray(images[0],'RGB').save('img.jpg')
  for x in range (n):
      latents = np.random.randn(number_of_images, *Gs.input_shapes[0][1:])
      labels = np.zeros([latents.shape[0]] + Gs.input_shapes[1][1:])
      rimages = Gs.run(latents, labels)
      rimages = np.clip(np.rint((rimages + 1.0) / 2.0 * 255.0), 0.0, 255.0).astype(np.uint8) 
      rimages = rimages.transpose(0, 2, 3, 1)
      # images.append(rimages)
      images= np.concatenate((images, rimages))

  zip_images=file_process_in_memory(images)
  zip_file_bytes_io = io.BytesIO()
  with ZipFile(zip_file_bytes_io, 'w') as zip_file:
    x=0
    for bytes_stream in zip_images:
        zip_file.writestr(str(uuid.uuid4())+".jpg", bytes_stream.getvalue())
        x+=1
  zip_file_bytes_io.seek(0)
  return zip_file_bytes_io

In [31]:
from flask import Flask,send_file,Response
from flask_cors import CORS
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
CORS(app)
run_with_ngrok(app) 
# @app.route("/")
# def home():
#     return "<h1>GFG is great platform to learn</h1>"
@app.route("/")
def generateFace():
   zip_object=generate(1);
   send_file("img.jpg")

@app.route("/zip")
def generateFaceZip():
   zip_object=generatezip(10,0);
   return Response(zip_object.getvalue(),
                        mimetype='application/zip',
                        headers={'Content-Disposition': 'attachment;filename=randomFace.zip'})


In [32]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://799f-35-193-108-70.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


/tensorflow-1.15.2/python3.7/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
127.0.0.1 - - [02/Nov/2021 11:56:51] "GET /zip HTTP/1.1" 200 -
127.0.0.1 - - [02/Nov/2021 12:01:09] "GET /zip HTTP/1.1" 200 -


In [ ]:
latents = np.random.RandomState(1000).randn(1, *Gs.input_shapes[0][1:])
labels = np.zeros([latents.shape[0]] + Gs.input_shapes[1][1:])
images = Gs.run(latents, labels)
images = np.clip(np.rint((images + 1.0) / 2.0 * 255.0), 0.0, 255.0).astype(np.uint8) 
images = images.transpose(0, 2, 3, 1)

In [ ]:
# images